# I analized Thanksgiving data to find pattern in dinner menus

My goal in this project is to be able to find patterns regarding Thanksgiving in general (gender, age, income, travelling distance) and more specific about dinner menus.
I would relly heavily on plots as it's the perfect method convey results data. A picture woth a thousand words.
Each row in the dataset represents a respondent feedback to specific questions.
Dataset can be found at this location:
https://github.com/fivethirtyeight/data/tree/master/thanksgiving-2015

In [ ]:
import pandas as pd
import numpy as np
import csv

data=pd.read_csv("thanksgiving-2015-poll-data.csv", encoding="Latin-1")

### Examining the data

In [ ]:
data.columns

1. Most of the column names are questions, and most of the column values are string responses to the questions. 
2. Most of the columns are categorical, as a survey respondent had to select one of a few options i.e "What is typically the main dish at your Thanksgiving dinner?"

### Rename columns

Columns that represent `side dishes` ; `deserts` and `pies` are too long so I will rename them

In [ ]:
def columns_rename(col_start_str,replace_str):
    for col in data.columns:
         if col.startswith(col_start_str):
            items=col.split("-")
            items[1].replace(" ","")
            data.rename(columns={col:str(replace_str)+items[1]}, inplace= True)
            #complete_fl.append(str("Side dish") +items[1])

# rename side dish; pie; desert columns
#columns_rename(col_start_str="Which of these side dishes", replace_str="Side dish")
#columns_rename(col_start_str="Which type of pie",replace_str="Pie")
#columns_rename(col_start_str="Which of these desserts",replace_str="Deserts")

### Check dataset dimensions

In [ ]:
data.columns

In [ ]:
data.shape

Dataset contains 1058 rows, and 65 columns

### Percentage of people celebrating Thanksgiving:

In [ ]:
# import plotly plot modules
# pie plot syntax with plotly and cufflinks:
# https://plot.ly/pandas/pie-charts/

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()
import plotly.graph_objs as go
import cufflinks as cf

# get % of people celebarting Thanksgiving 
celeb_tg=data["Do you celebrate Thanksgiving?"].value_counts().to_frame()

# to plot with cufflinks I need to have a df where & labels columns can't be in index
celeb_tg.reset_index(inplace=True)
celeb_tg.columns=['labels','values']

celeb_tg.iplot(kind='pie',labels='labels',values='values',pull=.2,hole=.2,title='Celebrate Thanksgiving',
               textposition='outside',textinfo='value+percent')

1. As we can see the overwhelming majority are celebrating Thanksgiving
2. 92.6% celebrate; 7,37% don't
3. We are only interested in people celebrating Thanksgiving so we are going to filter out those who don't

### Filtering out people that don't celebrate

In [ ]:
data=data[data["Do you celebrate Thanksgiving?"]=="Yes"]

### Figuring out menu patternhttps://plot.ly/python/pie-charts/s:
* what are the most common main dishes
* what are the most common pies
* what are the most common desers

### Main dish

In [ ]:
# python pie plot syntax without cufflinks
# https://plot.ly/python/pie-charts/

dish=data["What is typically the main dish at your Thanksgiving dinner?"].value_counts()
dish=dish.to_frame()
dish.reset_index(inplace=True)
dish.columns=['labels','values']

plot_data=[]
plot_data.append({
        'labels': dish['labels'],
        'values': dish['values'],
        'type': 'pie',
        'name': 'Starry Night',
        'hoverinfo':'label+percent+name',
        'hole': .4,
        'pull': .2
       })
layout = dict(autosize = True, title = 'Main dish preference in Thanksgiving')

fig = dict(data=plot_data, layout=layout)
iplot(fig, filename='pie-subplots')

88.2% prefer Turkey<br>
2.98% prefer Ham/Pork<br>
2.05% prefer Tofurkey<br>
1.23% prefer chicken<br>
1.13% prefer roast beef<br>
0.30% prefer Turducken<br>

### See what are other main dish preferences

In [ ]:
dish2=data['What is typically the main dish at your Thanksgiving dinner? - Other (please specify)'].value_counts()
dish2

There are a few cases where people choose a different dish<br>
Because of the lack of data I will not expore this deeper

### Most common pie, desert, side dish

The choice of pie or desert in Thanksgiving dinner is represented by multiple columns instead of one.<br>
My goal here is to find out the most common pies and deserts. So I need to get the value_counts for a particular "item" and also collapse these collumns into one.<br>
For this I will use 2 functions:<br>
    1. return_list <--> gets the value_counts for a particular cathegory:
    2. filter_null <--> help in collapsing these columns i.e will return a value only when != null
    
Also for this I will use 2 types of plots, bar and pie chart. Pie chart are good when you need to get a percentage from total. By defult a pie chart in plotly will show percent + values.If I only want to get top 10 most occurances by default pie chart will plot the data inaccuratly, because when it will get the total from a slice of the dataframe.
To get arround this I created another column that represent the percentage 

In [ ]:
def return_list(search_string):
    # create a list of columns for a specific food type i.e pie
    result_list=[c for c in data.columns if c.startswith(search_string)]
    
    # get counts where row values for a particular column !=null
    result = data[result_list].apply(pd.value_counts).fillna(0)
    return result

# return not null values
def filter_nnull(x):
    for i in x:
        if i!=0:
            return i

### Figuring Out What Pies People Eat

In [ ]:
# search for pie columns
search_string='Which type of pie'

# get pie counts
result=return_list(search_string)

# collapse the dataframe to a series by only returning != null values
result['Pie type %']=result.apply(filter_nnull,axis=1)
pie=result['Pie type %'].sort_values(ascending=False).to_frame().copy()

# plotly h barc chart syntax
# https://plot.ly/pandas/horizontal-bar-charts/

layout=dict(autosize=True,
            title='Most common pie in Thanksgiving',
            xaxis=dict(domain=[0.08, 1])
           )

pie.head(10).sort_values(by='Pie type %', ascending=True).iplot(kind='barh',barmode='nowmal', bargap=.8, filename='pie_distrib',layout=layout)

# change pie to a dataframe
pie_p=pie.reset_index()
pie_p.columns=('labels','values')

# get percentage
pie_p["%"]=round(pie_p['values']/pie_p['values'].sum()*100,2)
pie_p.head(10).iplot(kind='pie',labels='labels',values='%',pull=.2,hole=.2,title='Pie distribution Thanksgiving',
                     textposition='outside',textinfo='value')

### Pie distribution:
1. Pumpkin 31.59%
2. Apple 22.27%
3. Pecan 14.82%
4. Sweet Potato 6.59%
5. Chocolate 5.76%

### Figuring out what are the side dishes

In [ ]:
# search string for side dish columns
search_string='Which of these side dishes'

#get side dish counts
result=return_list(search_string)
result['Side dishes']=result.apply(filter_nnull,axis=1)
side=result['Side dishes'].sort_values(ascending=False).to_frame().copy()

layout=dict(autosize=True,
            title='Most common side dishes in Thanksgiving',
            xaxis=dict(domain=[0.2, 1])
           )

side.head(10).sort_values(by='Side dishes', ascending=True).iplot(kind='barh',barmode='nowmal', bargap=.8, filename='side_dish_distrib',layout=layout)

side_p=side.reset_index()
side_p.columns=['labels','values']
side_p["%"]=round(side_p['values']/side_p['values'].sum()*100,2)
side_p.head(10).iplot(kind='pie',labels='labels',values='%',pull=.2,hole=.2,title='Side dish distribution in Thanksgiving',
                     textposition='outside',textinfo='value')

### Side dish distribution
1. Mashed potatoes 16%
2. Rolls/biscuits 15%
3. Green beans/green bean casserole 13.43%
4. Yams/sweet potato casserole 12.36%
5. Corn 9.09%

### Deserts patterns

In [ ]:
desert_list=[]
columns=data.columns.tolist() 
for col in columns:
    # search for desert columns
    if col.startswith("Which of these desserts"):
        
        # split to obtain only the desert string
        items=col.split("-")
        items[1].replace(" ","")
        
        # rename row so as to contain only the desert name
        data.rename(columns={col:items[1]}, inplace= True)
        desert_list.append(items[1])
        
desert=data[desert_list].count()
desert.sort_values(inplace=True, ascending=False)

# convert desert series to df to plot a pie chart
desert_p=desert.to_frame()
desert_p.reset_index(inplace=True)
desert_p.columns=('lables','values')

#get percent
desert_p["%"]=round(desert_p['values']/desert_p['values'].sum()*100,2)

#
desert.iplot(kind='bar', xTitle='Desert', yTitle='Counts',margin=go.Margin(b=130),title='Most common deserts\n in Thanksgiving 2015',
             filename='cufflinks/income-bar-chart')
desert_p.head(10).iplot(kind='pie',labels='lables',values='%',pull=.2,hole=.2,title='Desert dish distribution in Thanksgiving',
                     textposition='outside',textinfo='value')
desert_p

### Age distribution on Thanksgiving

In [ ]:
age=data["Age"].value_counts().copy()
age_df=age.to_frame()
age_df.reset_index(inplace=True)
age_df.columns=['labels','values']

age.iplot(kind='bar', xTitle='Age', yTitle='Counts', title='Age distribution\n in Thanksgiving 2015',
             filename='cufflinks/categorical-bar-chart')
age_df.iplot(kind='pie',labels='labels' ,values='values',pull=.1,hole=.1,title='Celebrating Thanksgiving percentage',
            textposition='outside',textinfo='value+percent')
age_df

In [ ]:
def convert_age(column):
    if pd.isnull(column)==True:
        return None
    if column=="60+":
        return 60
    else:
        items=column.split("-")
        return(int(items[0]))
data["int_age"]=data["Age"].apply(convert_age)
data["int_age"].value_counts()
    

As we can see the values for each age group are pretty even. The mean is at 236.75 and standard deviation 37.29

# Household income 


In [ ]:
def sort_index(row):
    strip1=row.split(' ')[0].split(',')
    if strip1!='Prefer':
        strip2=strip1[0][1:].split(',')
        return strip2[0]
    else:
        return 'Prefer'

In [ ]:
income=data["How much total combined money did all members of your HOUSEHOLD earn last year?"].value_counts()
income.iplot(kind='bar', xTitle='Income', yTitle='Counts',margin=go.Margin(b=160), title='Income distribution\n in Thanksgiving 2015',
             filename='cufflinks/income-bar-chart')

income=income.to_frame()
income.reset_index(inplace=True)
income.columns=['labels','values']
labels=income['labels'].tolist()
values=income['values'].tolist()

income['sort_index']=income['labels'].apply(sort_index)
print(income[1:].sort_values(by='sort_index'))



income.iplot(kind='pie',labels='labels',values='values',pull=.1,hole=.1,title='Income distribution in',
               textposition='outside',textinfo='value+percent')


### Correlating Travel Distance And Income

     

### Rename Income & Travel column 

In [ ]:
# rename columns
data.rename(columns={"How much total combined money did all members of your HOUSEHOLD earn last year?":'Income'}, inplace= True)
data.rename(columns={"How far will you travel for Thanksgiving?":'Travel'}, inplace= True)

# get unique column values
orig_travel=data['Travel'].unique().tolist()
orig_travel

### Replace Travel columns values

Travel column values are to long. We are going to replace them with something shorter and meaningful.

In [ ]:
new_travel=['same town','few hours or less','no travel','several hours']
data.replace({'Travel' : 
                  { "Thanksgiving is local--it will take place in the town I live in" : "local", 
                    "Thanksgiving is out of town but not too far--it's a drive of a few hours or less" : "few hours", 
                    "Thanksgiving is happening at my home--I won't travel at all" : "no travel",
                    "Thanksgiving is out of town and far away--I have to drive several hours or fly":"several hours"}}
                    ,inplace=True)
data['Travel'].value_counts()

In [ ]:
data['Income'].head()

As we can the 'Income' column does not store categorical values. We need to define a function that will extract the lower income limit. This will impact accuracy. Values would be skewed to the lower limit.

In [ ]:
def extract_income(income_str):
    if pd.isnull(income_str):
        return None
    income_str = income_str.split(" ")[0]
    if income_str == "Prefer":
        return None
    income_str = income_str.replace(",", "")
    income_str = income_str.replace("$", "")
    return int(income_str)

In [ ]:
data["Income"] = data["Income"].apply(extract_income)

In [ ]:
data.groupby(['Income'])['Travel'].value_counts()

### Plot travel based on income

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.ioff();
import random

%matplotlib inline
print(plt.style.available)
plt.style.use('fivethirtyeight')

In [ ]:
inc_travel=data.loc[:,['Income','Travel']]
a=pd.pivot_table(inc_travel, index = ['Income'], columns=['Travel'], aggfunc = len)
a.plot(title='Corelating travel distange based on income',figsize=(14,5));

In [ ]:
fig , ax =plt.subplots(figsize=(14,7))
a.plot.bar(ax=ax, title='Travel distance based on income');

In [ ]:
inc_low_travel=inc_travel[inc_travel['Income']<150000]
a_low=inc_low_travel.pivot_table(inc_low_travel,index=['Income'],columns=['Travel'],aggfunc=len)
a_low
inc_low_travel.groupby(['Income'])['Travel'].value_counts()

In [ ]:
config_cf={'showLink': False,'displayModeBar': False}
a=data.groupby(['Have you ever attended a "Friendsgiving?"'])['int_age'].mean()
col=['#32ab60','#8cb8d6']
fig=a.reset_index().iplot(kind='pie',labels='Have you ever attended a "Friendsgiving?"',values='int_age',textinfo='value',
                    asFigure=True,colors=col,pull=0.02,hole=0.15,
                     title='Average age for people celebrating Friendsgiving ')
iplot(fig,config=config_cf)

In [ ]:
a=data.groupby(['Have you ever attended a "Friendsgiving?"'])['Income'].mean()
fig=a.reset_index().iplot(kind='pie',labels='Have you ever attended a "Friendsgiving?"',values='Income',textinfo='value',
                      colors=col,asFigure=True, pull=0.02,hole=0.15,
                     title='Average income for people celebrating Friendsgiving ')
iplot(fig,config=config_cf)

In [ ]:
data['US Region'].unique()

In [ ]:
data.columns

In [ ]:
def rename_col(col_str,list):
    nl=[]
    for item in list:
        v=str(item).split("-")
        print(v)
        v=col_str+v[1]
        nl.append(v)
        
    return nl

In [ ]:
col_str='Side'
new_cols=rename_col(col_str,data.columns[11:26].tolist())
print(new_cols)

In [ ]:
new_cols

In [ ]:
old_columns=data.columns[11:26].tolist()
old_columns
print(len(old_columns))

In [ ]:
old_columns

In [ ]:
new_cols
print(len(new_cols))

In [ ]:
#old_columns=data.columns[11:26].tolist()
data.rename(columns=dict(zip(old_columns, new_cols)),inplace=True)

In [ ]:
data.columns

In [ ]:
col_str='Pie'
new_cols=rename_col(col_str,data.columns[26:39].tolist())
old_columns=data.columns[26:39].tolist()
data.rename(columns=dict(zip(old_columns, new_cols)),inplace=True)

In [ ]:
data.columns

In [ ]:
col_to_keep=data.columns[11:26].tolist()
col_to_keep.append('US Region')
print(col_to_keep)
#test=
test=data.loc[:,col_to_keep]
a=test.pivot_table(index='US Region',columns=test.columns[1:15].tolist(),aggfunc=len)
#a
len(test.columns)
a

In [ ]:
data.loc[:,col_to_keep].groupby('US Region').count()

In [ ]:
col_to_keep2=data.columns[26:39].tolist()
col_to_keep2.append('US Region')
test=data.loc[:,col_to_keep2].groupby('US Region').count()

In [ ]:
test

In [ ]:
test.index.get_level_values(0)

In [ ]:
test.index.get_level_values(0)

In [ ]:
test.index[0]

In [ ]:
test.stack()

In [ ]:
test

In [ ]:
regions=data['US Region'].unique()